$\textbf{Code}$

## Libraries required

In [ ]:
# Importing packages
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import math
import pandas as pd
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
import torch.nn.functional as F

## Functions used

### Input prepartion functions

Convert trajectory tensor to our required format

In [ ]:
def trajectory_conversion(trajectory):
  input_trajectory = torch.zeros(len(trajectory), 3)
  for i in range(1, trajectory.size()[0]):
    del_x = trajectory[i][0]-trajectory[i-1][0]
    del_y = trajectory[i][1]-trajectory[i-1][1]
    del_z = trajectory[i][2]-trajectory[i-1][2]

    input_trajectory[i][0] = del_x

    input_trajectory[i][1] = del_y

    input_trajectory[i][2] = del_z

  return input_trajectory

In [ ]:
def create_inout_sequences(input_data, tw=100):
    inout_seq = []
    L = len(input_data)
    for i in range(L-tw):
        train_seq = input_data[i:i+tw].to(device)
        train_label = input_data[i+tw].to(device) #changed to correct error
        inout_seq.append((train_seq ,train_label))
    return inout_seq

### model

In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_size=3, hidden_layer_size=128, output_size=3):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size

        self.lstm = nn.LSTM(input_size, hidden_layer_size)

        self.linear = nn.Linear(hidden_layer_size, output_size)

        self.hidden_cell = (torch.zeros(1,1,self.hidden_layer_size, device=device),
                            torch.zeros(1,1,self.hidden_layer_size, device=device))

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq) ,1, -1), self.hidden_cell)
        lstm_out = F.relu(lstm_out.view(len(input_seq), -1))
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]

### training

In [ ]:
def train_model(net, train_inout_seq, lr=0.001, epochs=20, device=device):
  model = net.to(device)
  optimizer = torch.optim.Adam(model.parameters(), lr=lr)
  loss_function = nn.MSELoss()
  model.train()
  #print(model)
  for i in range(epochs):
    #optimizer.zero_grad()
    for seq, labels in train_inout_seq:
      optimizer.zero_grad()
      model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size, device=device), torch.zeros(1, 1, model.hidden_layer_size, device=device))

      pred = model(seq)

      single_loss = loss_function(pred, labels)
      single_loss.backward()
      optimizer.step()
    #optimizer.step()

  return model

In [ ]:
def getTrain_TestTensor(filename):
  data = pd.read_csv(filename)
  df = pd.DataFrame(data)
  df = df.drop(columns=['#date', 'ssh[m]', 'average_ssh[m]', 'geoid_height[m]', 'tide_height[cm]'])
  df['timestamp'] = pd.to_timedelta(df['time']).astype('timedelta64[s]').astype(int)
  df['relative_timestamp'] = df['timestamp']-df['timestamp'][0]
  df = df.drop(columns=['time', 'timestamp'])
  tl = int(len(df)*0.68)
  print(tl)
  testSize = int(len(df)) - tl
  print(testSize)

  trainTensor = torch.zeros(tl, 3).to(device)
  trainTensor[0][0] = df['latitude'][0]
  trainTensor[0][1] = df['longitude'][0]
  trainTensor[0][2] = df['flight_altitude[m]'][0]
  print(trainTensor.shape)
  #Remove unneccary points
  j = 0
  for i in range(1, tl):
    if(df['relative_timestamp'][i]!=df['relative_timestamp'][i-1]):
      j+=1
      trainTensor[j][0] = df['latitude'][i]
      trainTensor[j][1] = df['longitude'][i]
      trainTensor[j][2] = df['flight_altitude[m]'][i]
  trainTensor = trainTensor[trainTensor.abs().sum(dim=1) != 0]
  testTensor = torch.zeros(testSize, 3).to(device)
  testTensor[0][0] = df['latitude'][tl]
  testTensor[0][1] = df['longitude'][tl]
  testTensor[0][2] = df['flight_altitude[m]'][tl]
  j = 0
  for i in range(tl, len(df)):
    if(df['relative_timestamp'][i]!=df['relative_timestamp'][i-1]):
      j+=1
      testTensor[j][0] = df['latitude'][i]
      testTensor[j][1] = df['longitude'][i]
      testTensor[j][2] = df['flight_altitude[m]'][i]
  testTensor = testTensor[testTensor.abs().sum(dim=1) != 0]
  print(len(trainTensor), end = ' ')
  print(len(testTensor))
  return [trainTensor, testTensor]

### testing/predicting

In [ ]:
def predict_30(model, conv_tT, trainTensor, fut_pred=30, device=device):
  test_inputs = conv_tT[-train_window:]
  model.eval()
  for i in range(fut_pred):
    seq = test_inputs[-train_window:].to(device)
    with torch.no_grad():
      model.hidden = (torch.zeros(1, 1, model.hidden_layer_size, device=device), torch.zeros(1, 1, model.hidden_layer_size, device=device))
      Pred = model(seq)
      newObservation = torch.tensor([[Pred[0], Pred[1], Pred[2]]])      
      torch.cat((test_inputs, newObservation))
  X = []
  Y = []
  Z = []
  trainTensor[-1][0].item()
  trainTensor[-1][1].item()
  for x, y, z in test_inputs[-fut_pred:]:
    X.append(x)
    Y.append(y)
    Z.append(z)
    #print(x, y, z)
  X = np.asarray(X)
  Y = np.asarray(Y)
  Z = np.asarray(Z)
  X = np.cumsum(X)
  Y = np.cumsum(Y)
  Z = np.cumsum(Z)
  X_f = X+trainTensor[-1][0].item()
  Y_f = Y+trainTensor[-1][1].item()
  Z_f = Z+trainTensor[-1][2].item()
  #plt.plot(X_f, Y_f)
  #plt.show()
  #print(X_f)
  #print(Y_f)
  predicted = torch.zeros(fut_pred, 3).to(device)
  for i in range(fut_pred):
    predicted[i][0] = X_f[i].item()
    predicted[i][1] = Y_f[i].item()
    predicted[i][2] = Z_f[i].item()
  return [predicted, X_f, Y_f, Z_f]

In [ ]:
def Completed_trajectory(model, conv_tT, trainTensor, fut_pred, testTensor, plot=True, device=device, merge=False):
  X_f = np.asarray([])
  Y_f = np.asarray([])
  Z_f = np.asarray([])
  predicted_tensor = torch.zeros(1, 3).to(device)
  predicted_tensor[0] = trainTensor[-1]
  temp = trainTensor[:]
  for i in range(int(fut_pred/30)):
    pred = predict_30(model, conv_tT, temp, 30, device)
    temp = torch.cat((temp, pred[0]))
    predicted_tensor = torch.cat((predicted_tensor, pred[0]))
    X_f = np.concatenate((X_f, pred[1]), axis=0)
    Y_f = np.concatenate((Y_f, pred[2]), axis=0)
    Z_f = np.concatenate((Z_f, pred[3]), axis=0)
  if fut_pred%30!=0:
    pred = predict_30(model, conv_tT, temp, int(fut_pred%30), device)
    temp = torch.cat((temp, pred[0]))
    predicted_tensor = torch.cat((predicted_tensor, pred[0]))
    X_f = np.concatenate((X_f, pred[1]), axis=0)
    Y_f = np.concatenate((Y_f, pred[2]), axis=0)
    Z_f = np.concatenate((Z_f, pred[3]), axis=0)

  if plot:
    plt.plot(trainTensor[:, :1].cpu(), trainTensor[:, 2:].cpu(), label='train')
    plt.plot(testTensor[:, :1].cpu(), testTensor[:, 2:].cpu(), label='test')
    plt.xlabel('latitude')
    plt.ylabel('altitude')
    plt.legend()
    plt.show()

    plt.plot(trainTensor[:, :1].cpu(), trainTensor[:, 2:].cpu(), label='train')
    plt.plot(X_f, Z_f, label='predicted')
    plt.xlabel('latitude')
    plt.ylabel('altitude')
    plt.legend()
    plt.show()

    plt.plot(trainTensor[:, :1].cpu(), trainTensor[:, 2:].cpu(), label='train')
    plt.plot(testTensor[:, :1].cpu(), testTensor[:, 2:].cpu(), label='test')
    plt.plot(X_f, Z_f, label='predicted')
    plt.xlabel('latitude')
    plt.ylabel('altitude')
    plt.legend()
    plt.show()
  if(merge):
    return temp
  return predicted_tensor[1:]

# Main

In [ ]:
!unzip data.zip

Archive:  data.zip
   creating: data/
   creating: data/Dropped_Bomb/
  inflating: data/Dropped_Bomb/traj21.csv  
  inflating: data/Dropped_Bomb/traj22.csv  
  inflating: data/Dropped_Bomb/traj23.csv  
  inflating: data/Dropped_Bomb/traj24.csv  
  inflating: data/Dropped_Bomb/traj25.csv  
  inflating: data/Dropped_Bomb/traj26.csv  
  inflating: data/Dropped_Bomb/traj27.csv  
  inflating: data/Dropped_Bomb/traj28.csv  
  inflating: data/Dropped_Bomb/traj29.csv  
  inflating: data/Dropped_Bomb/traj30.csv  
   creating: data/Fighter_Jet/
  inflating: data/Fighter_Jet/traj51.csv  
  inflating: data/Fighter_Jet/traj52.csv  
  inflating: data/Fighter_Jet/traj53.csv  
  inflating: data/Fighter_Jet/traj54.csv  
  inflating: data/Fighter_Jet/traj55.csv  
  inflating: data/Fighter_Jet/traj56.csv  
  inflating: data/Fighter_Jet/traj57.csv  
  inflating: data/Fighter_Jet/traj58.csv  
  inflating: data/Fighter_Jet/traj59.csv  
  inflating: data/Fighter_Jet/traj60.csv  
   creating: data/Helicopter/

In [ ]:
def getTrain_TestTensor2(filename):
  rm = 52
  data = pd.read_csv(filename)
  df = pd.DataFrame(data)
  z = (df['alt']*1e5).rolling(rm+1).mean()
  tl = int((len(df)-rm)*0.68)
  print(tl)
  testSize = int(len(df)-rm) - tl
  print(testSize)

  trainTensor = torch.zeros(tl, 3).to(device)
  trainTensor[0][0] = df['lat'][rm]
  trainTensor[0][1] = df['lon'][rm]
  trainTensor[0][2] = z[rm]*1e5
  print(trainTensor.shape)
  #Remove unneccary points

  for i in range(1, tl):
    trainTensor[i][0] = df['lat'][rm+i]
    trainTensor[i][1] = df['lon'][rm+i]
    trainTensor[i][2] = z[rm+i]*1e5
  #trainTensor = trainTensor[trainTensor.abs().sum(dim=1) != 0]
  testTensor = torch.zeros(testSize, 3).to(device)
  testTensor[0][0] = df['lat'][rm+tl]
  testTensor[0][1] = df['lon'][rm+tl]
  testTensor[0][2] = z[rm+tl]*1e5
  j = 0
  for i in range(tl, len(df)-rm):
    testTensor[j][0] = df['lat'][rm+i]
    testTensor[j][1] = df['lon'][rm+i]
    testTensor[j][2] = z[rm+i]*1e5
    j+=1
  #testTensor = testTensor[testTensor.abs().sum(dim=1) != 0]
  #print(len(trainTensor), end = ' ')
  #print(len(testTensor))
  return [trainTensor, testTensor]

## Training plane_model

In [ ]:
# Train the first file
file1 = '/content/data/Plane/traj10.csv'
trainTensor, testTensor = getTrain_TestTensor2(file1)
print(trainTensor)
#plt.plot(trainTensor[:, :1].cpu().numpy(), trainTensor[:, 2:].cpu().numpy())
#plt.plot(testTensor[:, :1].cpu().numpy(), testTensor[:, 2:].cpu().numpy())
#plt.show()

train_window = 50
testSize = len(testTensor)
conv_tT = trajectory_conversion(trainTensor)
train_inout_seq = create_inout_sequences(conv_tT, train_window)
net = LSTM()
tm = train_model(net, train_inout_seq, epochs=15) #training code
torch.save(tm.state_dict(), 'plane_model_0.pth')

In [ ]:
import glob
n = 0
for f in glob.iglob(r'/content/data/Plane/traj?.csv'):
  print(f)
  if f=='/content/data/Plane/traj1.csv':
    continue
  trainingTensor, testingTensor = getTrain_TestTensor2(f)
  train_window = 50
  conv_tT = trajectory_conversion(trainTensor)
  train_inout_seq = create_inout_sequences(conv_tT, train_window)
  name = 'plane_model_'+str(n)+'.pth'
  net = LSTM()
  net.load_state_dict(torch.load(name))
  tm = train_model(net, train_inout_seq, epochs=15) #training code
  n+=1
  name = 'plane_model_'+str(n)+'.pth'
  torch.save(tm.state_dict(), name)

## Training helicopter_model

In [ ]:
# Train the first file
file1 = '/content/data/Helicopter/traj20.csv'
trainTensor, testTensor = getTrain_TestTensor2(file1)
print(trainTensor)
#plt.plot(trainTensor[:, :1].cpu().numpy(), trainTensor[:, 2:].cpu().numpy())
#plt.plot(testTensor[:, :1].cpu().numpy(), testTensor[:, 2:].cpu().numpy())
#plt.show()

train_window = 50
testSize = len(testTensor)
conv_tT = trajectory_conversion(trainTensor)
train_inout_seq = create_inout_sequences(conv_tT, train_window)
net = LSTM()
tm = train_model(net, train_inout_seq, epochs=15) #training code
torch.save(tm.state_dict(), 'helicopter_model_0.pth')

In [ ]:
import glob
n = 0
for f in glob.iglob(r'/content/data/Helicopter/traj1?.csv'):
  print(f)
  if f=='/content/data/Helicopter/traj11.csv':
    continue
  trainingTensor, testingTensor = getTrain_TestTensor2(f)
  train_window = 50
  conv_tT = trajectory_conversion(trainTensor)
  train_inout_seq = create_inout_sequences(conv_tT, train_window)
  name = 'helicopter_model_'+str(n)+'.pth'
  net = LSTM()
  net.load_state_dict(torch.load(name))
  tm = train_model(net, train_inout_seq, epochs=15) #training code
  n+=1
  name = 'helicopter_model_'+str(n)+'.pth'
  torch.save(tm.state_dict(), name)

## Training DB_model

In [ ]:
# Train the first file
file1 = '/content/data/Dropped_Bomb/traj30.csv'
trainTensor, testTensor = getTrain_TestTensor2(file1)
print(trainTensor)
#plt.plot(trainTensor[:, :1].cpu().numpy(), trainTensor[:, 2:].cpu().numpy())
#plt.plot(testTensor[:, :1].cpu().numpy(), testTensor[:, 2:].cpu().numpy())
#plt.show()

train_window = 50
testSize = len(testTensor)
conv_tT = trajectory_conversion(trainTensor)
train_inout_seq = create_inout_sequences(conv_tT, train_window)
net = LSTM()
tm = train_model(net, train_inout_seq, epochs=15) #training code
torch.save(tm.state_dict(), 'DB_model_0.pth')

In [ ]:
import glob
n = 0
for f in glob.iglob(r'/content/data/Dropped_Bomb/traj2?.csv'):
  print(f)
  if f=='/content/data/Dropped_Bomb/traj21.csv':
    continue
  trainingTensor, testingTensor = getTrain_TestTensor2(f)
  train_window = 50
  conv_tT = trajectory_conversion(trainTensor)
  train_inout_seq = create_inout_sequences(conv_tT, train_window)
  name = 'DB_model_'+str(n)+'.pth'
  net = LSTM()
  net.load_state_dict(torch.load(name))
  tm = train_model(net, train_inout_seq, epochs=15) #training code
  n+=1
  name = 'DB_model_'+str(n)+'.pth'
  torch.save(tm.state_dict(), name)

## Training Missile_model

In [ ]:
# Train the first file
file1 = '/content/data/Missile/traj40.csv'
trainTensor, testTensor = getTrain_TestTensor2(file1)
print(trainTensor)
plt.plot(trainTensor[:, :1].cpu().numpy(), trainTensor[:, 2:].cpu().numpy())
plt.plot(testTensor[:, :1].cpu().numpy(), testTensor[:, 2:].cpu().numpy())
plt.show()

train_window = 50
testSize = len(testTensor)
conv_tT = trajectory_conversion(trainTensor)
train_inout_seq = create_inout_sequences(conv_tT, train_window)
net = LSTM()
tm = train_model(net, train_inout_seq, epochs=15) #training code
torch.save(tm.state_dict(), 'missile_model_0.pth')

In [ ]:
import glob
n = 0
for f in glob.iglob(r'/content/data/Missile/traj3?.csv'):
  print(f)
  if f=='/content/data/Missile/traj31.csv':
    continue
  trainingTensor, testingTensor = getTrain_TestTensor2(f)
  train_window = 50
  conv_tT = trajectory_conversion(trainTensor)
  train_inout_seq = create_inout_sequences(conv_tT, train_window)
  name = 'missile_model_'+str(n)+'.pth'
  net = LSTM()
  net.load_state_dict(torch.load(name))
  tm = train_model(net, train_inout_seq, epochs=15) #training code
  n+=1
  name = 'missile_model_'+str(n)+'.pth'
  torch.save(tm.state_dict(), name)

## Training FJ_model

In [ ]:
# Train the first file
file1 = '/content/data/Fighter_Jet/traj60.csv'
trainTensor, testTensor = getTrain_TestTensor2(file1)
print(trainTensor)
#plt.plot(trainTensor[:, :1].cpu().numpy(), trainTensor[:, 2:].cpu().numpy())
#plt.plot(testTensor[:, :1].cpu().numpy(), testTensor[:, 2:].cpu().numpy())
#plt.show()

train_window = 50
testSize = len(testTensor)
conv_tT = trajectory_conversion(trainTensor)
train_inout_seq = create_inout_sequences(conv_tT, train_window)
net = LSTM()
tm = train_model(net, train_inout_seq, epochs=15) #training code
torch.save(tm.state_dict(), 'FJ_model_0.pth')

In [ ]:
import glob
n = 0
for f in glob.iglob(r'/content/data/Fighter_Jet/traj5?.csv'):
  print(f)
  if f=='/content/data/Fighter_Jet/traj51.csv':
    continue
  trainingTensor, testingTensor = getTrain_TestTensor2(f)
  train_window = 50
  conv_tT = trajectory_conversion(trainTensor)
  train_inout_seq = create_inout_sequences(conv_tT, train_window)
  name = 'FJ_model_'+str(n)+'.pth'
  net = LSTM()
  net.load_state_dict(torch.load(name))
  tm = train_model(net, train_inout_seq, epochs=15) #training code
  n+=1
  name = 'FJ_model_'+str(n)+'.pth'
  torch.save(tm.state_dict(), name)

## save models in models.zip file

In [36]:
!mkdir models

In [37]:
import shutil
for f in glob.iglob(r'/content/*.pth'):
  shutil.move(f, '/content/models/'+f[9:])
  print(f[9:], 'moved')

missile_model_0.pth moved
FJ_model_7.pth moved
missile_model_4.pth moved
FJ_model_0.pth moved
FJ_model_4.pth moved
FJ_model_6.pth moved
FJ_model_8.pth moved
missile_model_6.pth moved
missile_model_2.pth moved
missile_model_7.pth moved
FJ_model_5.pth moved
FJ_model_3.pth moved
missile_model_3.pth moved
FJ_model_1.pth moved
missile_model_1.pth moved
FJ_model_2.pth moved
FJ_1.pth moved
missile_model_8.pth moved
missile_model_5.pth moved


In [ ]:
!zip -r /content/models.zip /content/models